In [1]:

import numpy as np
from datasets import Dataset, load_dataset
import datasets
import pandas as pd
from datasets import Dataset
import random
import numpy as np
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer
import torch
from transformers import BitsAndBytesConfig
from torch import nn
from typing import TYPE_CHECKING, Any, Callable, Dict, List, Optional, Tuple, Union
import torch.nn.functional as F


/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.10/site-packages/torch/cuda/__init__.py:654: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
dataset = load_dataset('csv', data_files='data/paradetox.tsv', delimiter='\t', split='train')

tokenizer = AutoTokenizer.from_pretrained('model')

def preprocess_function(examples):
    text = examples['toxic']
    examples['input_ids'] = tokenizer("paraphrase: " + text, max_length=128, truncation=True)['input_ids']
    # example['labels'] = tokenizer(text_target=neutral1, max_length=max_target_length, truncation=True)['input_ids']
    # new_examples['labels'].append(labels['input_ids'])
    return examples

# dataset = dataset.map(preprocess_function, remove_columns=['id','toxic','severe_toxic','obscene','threat','insult','identity_hate'])

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
print(dataset)

dataset = dataset.map(preprocess_function, remove_columns=['neutral1', 'neutral2', 'neutral3']).select(range(500))
print(dataset)
# dataset = dataset.map(preprocess_function, remove_columns=['id','toxic','severe_toxic','obscene','threat','insult','identity_hate'])

Dataset({
    features: ['toxic', 'neutral1', 'neutral2', 'neutral3'],
    num_rows: 11927
})


Map:   0%|          | 0/11927 [00:00<?, ? examples/s]

Dataset({
    features: ['toxic', 'input_ids'],
    num_rows: 500
})


In [4]:
model_name = 'model'

my_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


input_texts = []
for i in range(len(dataset)):
    new_inputs = my_model.generate(torch.tensor([dataset[i]['input_ids']]).to(my_model.device))
    words = tokenizer.batch_decode(new_inputs, skip_special_tokens=True)
    input_texts.append(dataset[i]['toxic'])
    input_texts.append(words[0])
    # result = judge.judge(words)
    # if result==0:
    #     count += 1


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [5]:
old = input_texts[0::2]
new = input_texts[1::2]

In [6]:
import csv

# with open('test_result_two/model_with_dpo.csv', 'w', newline='') as file:
with open('model_result.csv', 'w', newline='') as file:
# with open('test_result_two/based_model.csv', 'w', newline='') as file:
    
    writer = csv.writer(file)
    writer.writerow(['old', 'new'])
    # Optional: If you want each string in its own row
    for i in range(len(old)):  # Loop through the length of the first column

        writer.writerow([old[i], new[i]])


In [7]:

import numpy as np
from datasets import Dataset, load_dataset
import datasets
import pandas as pd
from datasets import Dataset
import random
import numpy as np
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer
import torch
from transformers import BitsAndBytesConfig
from torch import nn
from typing import TYPE_CHECKING, Any, Callable, Dict, List, Optional, Tuple, Union
import torch.nn.functional as F


In [8]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch

# load tokenizer and model weights
tokenizer = RobertaTokenizer.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')
model = RobertaForSequenceClassification.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')

# prepare the input
# batch = tokenizer.encode('you are amazing', return_tensors='pt')



/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:484: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location=map

In [16]:
dataset = load_dataset('csv', data_files='model_result.csv', split='train')

In [17]:
print(dataset)

Dataset({
    features: ['old', 'new'],
    num_rows: 500
})


In [18]:
sum = 0

for i in range(len(dataset['new'])):
    batch = tokenizer.encode(dataset['new'][i], return_tensors='pt')
    result = model(batch)
    if torch.softmax(result.logits[0], dim=0)[0] > 0.8:
        sum += 1


In [19]:
print(sum/500)

0.474


In [14]:
batch = tokenizer.encode(dataset['old'][0], return_tensors='pt')

In [15]:
result = model(batch)

In [16]:
print(result)

SequenceClassifierOutput(loss=None, logits=tensor([[-2.3960,  2.3409]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [11]:
print(torch.softmax(result.logits[0], dim=0))

tensor([0.5105, 0.4895], grad_fn=<SoftmaxBackward0>)
